# 基于MindSpore实现softmax

In [1]:
import sys
sys.path.append('..')

In [2]:
import mindspore
from mindspore import nn
from mindspore import ops

给定一个矩阵X，我们可以对所有元素求和

In [3]:
X = mindspore.Tensor([[1.0, 2.0, 3.0], [4.0, 5.0, 6.0]], mindspore.float32)
X.sum(0, keepdims=True), X.sum(1, keepdims=True)

(Tensor(shape=[1, 3], dtype=Float32, value=
 [[ 5.00000000e+00,  7.00000000e+00,  9.00000000e+00]]),
 Tensor(shape=[2, 1], dtype=Float32, value=
 [[ 6.00000000e+00],
  [ 1.50000000e+01]]))

实现**softmax**: $${softmax}(X)_{ij}=\frac{exp(X_{ij})}{\Sigma_{k}{exp(X_{ik})}}.$$


In [4]:
import mindspore.ops as ops

def softmax(X):
    X_exp = ops.exp(X)
    partition = X_exp.sum(1, keepdims=True)
    return X_exp / partition

我们将每个元素变成一个非负数。此外，依据概率原理，每行总和为1

In [5]:
import numpy as np

X = mindspore.Tensor(np.random.normal(0, 1, (2, 5)), mindspore.float32)
X_prob = softmax(X)
X_prob, X_prob.sum(1)

(Tensor(shape=[2, 5], dtype=Float32, value=
 [[ 1.78480148e-01,  2.09095493e-01,  1.64532140e-01,  2.51835883e-01,  1.96056336e-01],
  [ 1.69591725e-01,  9.90628228e-02,  1.69651821e-01,  2.95120001e-01,  2.66573548e-01]]),
 Tensor(shape=[2], dtype=Float32, value= [ 1.00000000e+00,  9.99999881e-01]))

In [6]:
import mindspore.nn as nn
from mindspore import Tensor, Parameter
from mindspore.common.initializer import initializer, Zero, Normal

class Net(nn.Cell):
    def __init__(self, num_inputs, num_outputs):
        super().__init__()
        self.W = Parameter(initializer(Normal(0.01, 0), (num_inputs, num_outputs), mindspore.float32))
        self.b = Parameter(initializer(Zero(), num_outputs, mindspore.float32))

    def construct(self, X):
        return softmax(ops.matmul(X.reshape((-1, self.W.shape[0])), self.W) + self.b)

创建一个数据样本y_hat，其中包含2个样本在3个类别的预测概率，及它们对应的标签y。 使用y作为y_hat中概率的索引

In [7]:
y = mindspore.Tensor([0, 2], mindspore.int32)
y_hat = mindspore.Tensor([[0.1, 0.3, 0.6], [0.3, 0.2, 0.5]], mindspore.float32)
y_hat[[0, 1], y]

Tensor(shape=[2], dtype=Float32, value= [ 1.00000001e-01,  5.00000000e-01])

实现交叉熵损失函数

In [8]:
import mindspore.numpy as mnp

def cross_entropy(y_hat, y):
    return -mnp.log(y_hat[mnp.arange(y_hat.shape[0]), y])

cross_entropy(y_hat, y)

Tensor(shape=[2], dtype=Float32, value= [ 2.30258512e+00,  6.93147182e-01])

将预测类别与真实y元素进行比较

In [9]:
def accuracy(y_hat, y):  
    """计算预测正确的数量。"""
    if len(y_hat.shape) > 1 and y_hat.shape[1] > 1:
        y_hat = y_hat.argmax(axis=1)
    cmp = y_hat.asnumpy() == y.asnumpy()
    return float(cmp.sum())

accuracy(y_hat, y) / len(y)

0.5